# Proyecto: Visualización 3D de modelo de exposición

## Descripción del proyecto

Este proyecto tiene como objetivo investigar e implementar metodologías novedosas de visualización de datos de exposición a distintas escalas espaciales, que permitan representar y comunicar de manera atractiva la información contenida en ellos, como ubicación, altura, densidad, materialidad, tipo de uso, edad, tipología estructural, etc.

## Datos disponibles

- Modelo de exposición: Los datos principales de este proyecto corresponde al modelo de exposición producto de la publicación "Development of national and local exposure models of residential structures in Chile". Este modelo se constituye de 1 archivo .txt para cada región, que incluye un catastro a nivel de bloque censal para 18 tipologías distintas de estructuras. 

- Datos de salud pública: Se obtienen directamente desde el MINSAL. Estos datos presentan el desafío de que son actualizados constantemente, por lo que su url es variable. Se debe pensar en como solucionar este problema.

- Datos de educación: Obtenidos desde la página de datos abiertos del [MINEDUC](http://datosabiertos.mineduc.cl/). Estos presentan el desafío de que su formato es en .rar, formato que es dificil de abordar desde la automatización, pues WinRar es un producto de licencia y no open-source.

- Datos de cartografía censal: Obtenidos desde la página del [censo](http://www.censo2017.cl/servicio-de-mapas/). Presenta el mismo problema: vienen en formato .rar.


## Tratamiento de los datos.

### Modelo de exposición. 

Los modelos de exposición se encuentran en formato .txt, y deben ser transformados a un formato adecuado para el trabajo de datos geoespaciales. Esto se realiza a través de pandas y los métodos implementados en geopandas, que nos permiten obtener la geometría directamente desde un dataset.